In [1]:
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
import joblib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
#doginfo.csv파일 데이터를 pandas를 이용해 읽어옵니다.
dog_data=pd.read_csv("doginfo.csv")
dog_train=pd.read_csv("testDog.csv")
kindCd=pd.read_csv("kindCd.csv")

dog_data = dog_data.dropna(axis=0)
dog_train = dog_train.dropna(axis=0)
kindCd_data = kindCd.dropna(axis=0)

print(dog_data)
print(dog_train)
print(kindCd_data)

       kindNum neuterYn sexCd  weight  noticeDays  age2  processState
0        128.0        N     F    7.46          10    12             0
1        114.0        N     M    7.00          14     1             1
2        114.0        U     M    4.50          11     2             0
3         67.0        N     M   10.00           8     1             0
4        114.0        N     M    6.00           8     4             0
...        ...      ...   ...     ...         ...   ...           ...
22787    114.0        N     M    1.00          10     0             1
22788    114.0        N     M    1.00          10     0             1
22789    114.0        N     M    1.00          10     0             1
22790    128.0        U     F    6.00          12     3             0
22791    114.0        N     M    3.50          10     0             0

[22777 rows x 7 columns]
     kindNum neuterYn sexCd  weight  noticeDays  age2  processState
0      128.0        N     M     0.8          10     0             

In [3]:
kindCd = np.array(kindCd_data, dtype = np.float64)
kindCd_train=np.array(kindCd_data, dtype=np.float64)

kindCd = kindCd.reshape(177)
kindCd_train=kindCd_train.reshape(177)

print(kindCd)
print(kindCd.shape)

[ 54.  56.  55. 118. 115.  37.  81. 204.  83.  82.  38.  39.  40.  43.
  42. 153.  41. 120. 155.  69.  71. 142.  93. 167.  70. 166.  94. 121.
 152.  73. 146.  72. 159.  76.  75.  79.  78.  77.  74.  80. 114. 133.
  12.  17.  15. 164. 157. 148.  16.  20.  21.  22.  24. 208.  23.  26.
  27. 169.  25.  19.  13.  18.  14. 162.  85.  96.  95.   1.  34. 104.
  31.  99. 122. 123.  97. 132. 105. 154. 124. 100. 103. 151. 139. 101.
 102.  98. 136. 202. 160. 203.   8. 131.   9. 119. 150. 210.  57.  58.
  59.   6.   4.   7.   5. 143.  11.  10. 137.  84. 163. 112. 113. 149.
 211. 110. 205. 108. 109.  60.  46.  47.  44.  45.  53.  62.  61.  52.
 165.  51. 156. 129.  67.  35.  33.  32. 158. 144.  30.  29.  64. 207.
  28.   2.  68. 125. 141. 145.  36.  66.  65.  63. 140. 107. 106. 209.
  86.  88.  90.  87. 138.  89. 126. 127. 128.  91.   3. 161.  50. 168.
  49. 147.  92.  48. 135. 206. 130. 134. 111.]
(177,)


In [4]:
# kindNum을 원핫 인코딩
kindCd = pd.concat((pd.get_dummies(dog_data.kindNum, columns=kindCd), pd.DataFrame(columns=kindCd))).fillna(0)
# kindCd

# 학습데이터에서 kindNum 열을 삭제한 후, 원핫 인코딩된 kindCd를 붙임
dog_data.drop(['kindNum'], axis='columns', inplace=True)
dog_data = pd.concat([dog_data, kindCd], axis=1)


# kindNum을 원핫 인코딩
kindCd_train = pd.concat((pd.get_dummies(dog_train.kindNum, columns=kindCd_train), pd.DataFrame(columns=kindCd_train))).fillna(0)

# 테스트데이터에서 kindNum 열을 삭제한 후, 원핫 인코딩된 kindCd를 붙임
dog_train.drop(['kindNum'], axis='columns', inplace=True)
dog_train = pd.concat([dog_train, kindCd_train], axis=1)

print(dog_data)
print(dog_train)

      neuterYn sexCd  weight  noticeDays  age2  processState  1.0  2.0  3.0  \
0            N     F    7.46          10    12             0    0    0    0   
1            N     M    7.00          14     1             1    0    0    0   
2            U     M    4.50          11     2             0    0    0    0   
3            N     M   10.00           8     1             0    0    0    0   
4            N     M    6.00           8     4             0    0    0    0   
...        ...   ...     ...         ...   ...           ...  ...  ...  ...   
22787        N     M    1.00          10     0             1    0    0    0   
22788        N     M    1.00          10     0             1    0    0    0   
22789        N     M    1.00          10     0             1    0    0    0   
22790        U     F    6.00          12     3             0    0    0    0   
22791        N     M    3.50          10     0             0    0    0    0   

       4.0  ...  202.0  203.0  204.0  205.0  206.0 

In [5]:
def dummy_data(data, columns):
    for column in columns:
        data = pd.concat([data, pd.get_dummies(data[column], prefix = column)], axis = 1)
        data = data.drop(column, axis = 1)
    return data

In [6]:
dummy_columns = ["neuterYn", "sexCd"]
data = dummy_data(dog_data, dummy_columns)
train_data = dummy_data(dog_train, dummy_columns)

print(data.head())
print(train_data.head())

data = np.array(data, dtype = np.float64)
train_data = np.array(train_data, dtype = np.float64)

   weight  noticeDays  age2  processState  1.0  2.0  3.0  4.0  5.0  6.0  ...  \
0    7.46          10    12             0    0    0    0    0    0    0  ...   
1    7.00          14     1             1    0    0    0    0    0    0  ...   
2    4.50          11     2             0    0    0    0    0    0    0  ...   
3   10.00           8     1             0    0    0    0    0    0    0  ...   
4    6.00           8     4             0    0    0    0    0    0    0  ...   

   208.0  209.0  210.0  211.0  neuterYn_N  neuterYn_U  neuterYn_Y  sexCd_F  \
0      0      0      0      0           1           0           0        1   
1      0      0      0      0           1           0           0        0   
2      0      0      0      0           0           1           0        0   
3      0      0      0      0           1           0           0        0   
4      0      0      0      0           1           0           0        0   

   sexCd_M  sexCd_Q  
0        0        0  
1     

In [7]:
a = data[:, :3]
b = data[:, 4:]

#numpy 배열에서 데이터 변화요인(kindCd, neuterYn, sexCd, weight, noticeDays, age2)으로 사용할 데이터를 뽑아냅니다.
xData = np.concatenate([a, b], axis = 1)

a=train_data[:, :3]
b=train_data[:, 4:]

testX=np.concatenate([a, b], axis = 1)

In [8]:
#numpy배열에서 결과(입양여부)로 사용할 데이터를 뽑아냅니다.
yData=data[:,[3]]
testY=train_data[:,[3]]

In [9]:
def model_performance(y_test, y_pred):    
    print('confusion matrix')
    print(metrics.confusion_matrix(y_test, y_pred))
    print('accuracy : {}'.format(metrics.accuracy_score(y_test, y_pred)))
    print('precision : {}'.format(metrics.precision_score(y_test, y_pred)))
    print('recall : {}'.format(metrics.recall_score(y_test, y_pred)))
    print('F1 : {}'.format(metrics.f1_score(y_test, y_pred)))

In [10]:
from sklearn.model_selection import GridSearchCV
params = { 'n_estimators' : [10, 100],
           'max_depth' : [6, 8, 10, 12],
           'min_samples_leaf' : [8, 12, 18],
           'min_samples_split' : [8, 16, 20]
            }

# RandomForestClassifier 객체 생성 후 GridSearchCV 수행
rf_clf = RandomForestClassifier(random_state = 0)
grid_cv = GridSearchCV(rf_clf, param_grid = params, scoring='accuracy',n_jobs = -1, verbose=1)
grid_cv.fit(xData, yData)

print('최적 하이퍼 파라미터: ', grid_cv.best_params_)
print('최고 예측 정확도: {:.4f}'.format(grid_cv.best_score_))

Fitting 5 folds for each of 72 candidates, totalling 360 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   39.8s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:  3.3min
[Parallel(n_jobs=-1)]: Done 360 out of 360 | elapsed:  7.0min finished
C:\Users\auddu\anaconda3\envs\auddus\lib\site-packages\sklearn\model_selection\_search.py:765: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)


최적 하이퍼 파라미터:  {'max_depth': 12, 'min_samples_leaf': 8, 'min_samples_split': 20, 'n_estimators': 100}
최고 예측 정확도: 0.6832


In [35]:
model = RandomForestClassifier()
rf_model=model.fit(xData, yData)
y_pred = rf_model.predict(testX)
model_performance(testY, y_pred)

C:\Users\auddu\anaconda3\envs\auddus\lib\site-packages\ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


confusion matrix
[[180 128]
 [109 193]]
accuracy : 0.6114754098360655
precision : 0.6012461059190031
recall : 0.6390728476821192
F1 : 0.6195826645264848


In [80]:
rf_model.predict_proba(testX[:6])

array([[0.        , 1.        ],
       [0.64494874, 0.35505126],
       [0.5       , 0.5       ],
       [0.44796212, 0.55203788],
       [0.90261905, 0.09738095],
       [0.90261905, 0.09738095]])

In [81]:
joblib.dump(rf_model, 'randomforest.pkl')

['randomforest.pkl']